In [1]:
#!pip install accelerate
import torch
import guidance

In [2]:
llama7b = guidance.llms.transformers.LLaMA("/Users/titus.w/Code/wjt/llama-7b")
gpt3 = guidance.llms.OpenAI("text-davinci-003")
gpt4 = guidance.llms.OpenAI("gpt-4") 

You are using the legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This means that tokens that come after special tokens will not be properly handled. We recommend you to read the related pull request available at https://github.com/huggingface/transformers/pull/24565


Loading checkpoint shards:   0%|          | 0/33 [00:00<?, ?it/s]

### gen后面要声明一个位置变量，用于存储生成的数据

In [5]:
guidance.llm = llama7b
program = guidance("""This is a sentence about {{gen "completion" stop="."}}""")
executed_program = program()

Stop program This is a sentence about the

In [6]:
executed_program["completion"]
#llama7b模型也太捞了

'the'

In [8]:
out = guidance("""This is a sentence about {{gen stop="."}}""")()
out.variables()

Stop program This is a sentence about the

{'llm': <guidance.llms.transformers._llama.LLaMA at 0x16b64c0a0>,
 'logging': False,
 '@raw_prefix': 'This is a sentence about {{!--GMARKER_START_gen$&#123;&#123;gen stop="."&#125;&#125;$--}}the{{!--GMARKER_END_gen$$--}}'}

### stop参数是生成到什么时候停止

In [10]:
#可以使用点符号来设置对象或字典元素的属性
out = guidance("""This is a sentence about {{gen 'obj.completion' stop="."}}""")(obj={})
out["obj"]

Stop program This is a sentence about the

{'completion': 'the'}

In [12]:
out = guidance("""This is a sentence about {{gen 'text' stop=[" of", " a"]}}""")()

Stop program This is a sentence about the

### stop_regex参数可以设置停止的正则表达式
### save_stop_text 参数会使 gen 命令保存导致停止生成的文本。

In [2]:
import guidance
guidance.llm = guidance.llms.OpenAI("text-davinci-003", caching=False)

program = guidance("""Please solve the following word problem and call a calcuator with CALC(EQUATION) = ANSWER whenever you need to compute equations. For example: CALC((4+3) * 2) = 14.
Problem: {{problem}}
Reason step by step: {{gen 'text' stop_regex="CALC\\(.*?\\) =" max_tokens=100 save_stop_text=True}}""")
program(problem="Joe has ten apples and needs run 5 tests on each apple, if each test takes 7 minutes how long will this take Joe?")

Stop program Please solve the following word problem and call a calcuator with CALC(EQUATION) = ANSWER whenever you need to compute equations. For example: CALC((4+3) * 2) = 14.
Problem: {{problem}} 
Reason step by step: {{gen 'text' stop_regex="CALC\(.*?\) =" max_tokens=100 save_stop_text=True}}

In [4]:
out = guidance("""This is a sentence about {{gen 'text' max_tokens=10}}""")()
out["text"]

Stop program This is a sentence about {{gen 'text' max_tokens=10}}

KeyError: 'text'

### n 参数控制着一次批处理要执行多少代。如果 n > 1，那么只有第一次完成的结果会被用于未来的连接，其余的结果都会存储在变量中。

In [17]:
out = guidance("""This is a sentence about {{gen 'text' n=5 max_tokens=5 temperature=1.0}}""")()
out["text"]

Stop program This is a sentence about {{gen 'text' n=5 max_tokens=5 temperature=1.0}}

KeyError: 'text'

### temperature参数控制采样温度，并直接传递给 LLM。温度为零时，每次生成的文本都将相同。温度不为零时，每次生成的文本都会不同

In [ ]:
out = guidance("""This is a sentence about {{gen 'text' n=5 max_tokens=5}}""")()
out["text"]

In [ ]:
out = guidance("""This is a sentence about {{gen 'text' n=5 max_tokens=5 temperature=1.0}}""")()
out["text"]

### top_p 参数控制抽样使用的概率空间比例,有在温度值不为零的情况下，设置 top_p才有意义，越高采样低概率词块的可能性越大。

In [ ]:
out = guidance("""This is a sentence about {{gen 'text' n=5 max_tokens=5 top_p=0.4 temperature=1.0}}""")()
out["text"]

In [ ]:
# logprobs参数大于0，将返回token的对数概率。
# pattern参数也是正则表达式。
# hidden参数为ture就是多次生成中，后一次可以包含前一次的生成内容。
# parse参数为 True 时，生成的结果将作为指导模板进行递归解析。
# list_append 参数为 True 时，生成的结果将附加到 gen 参数给出的列表中。
# Guidance 中使用角色标记。虽然每个聊天模型都可以定义自己的特殊标记，但角色标记可以在所有聊天模型中使用。使用 {{#role "role_name"}}...{{/role}} 命令可以指定一个角色。由于 ChatGPT 使用三种角色，我们为每种角色提供了方便的命令：{{#user}}...{{/user}}、{{#assistant}}...{{/assistant}} 和 {{#system}}...{{/system}} 。这些命令分别等同于 {{#role "user"}}...{{/role}}, {{#role "assistant"}}...{{/role}} 和 {{#role "system"}}...{{/role}} 。
# ~字符是一种标准语法，用于表示命令前或命令后的空白区域（取决于 ~ 的位置）应被忽略。
# {{~！ xxx ~}}是注释。
# {{~#each examples}} {{~/each}} 循环语句，循环examples中每一个列表，this.属性是指当前列表的属性，这里要注意循环套循环。
# 蓝色是example文本，绿色是大语言模型生成文本。

In [ ]:
out = guidance("""Write a story about a person and use \{{character_name}} whenever you need to write their name:
STORY
{{gen 'story' max_tokens=100 parse=True}}""", llm=gpt3, character_name="Jill")()

### 注意，我们使用 \{{}} 来转义 {{}}，这样就可以用它来教模型模板语法了.

In [5]:
import guidance
guidance.llm = guidance.llms.OpenAI("text-davinci-003")
guidance("""Write a story about a person and use \{{character_name}} whenever you need to write their name:
STORY
{{parse (gen max_tokens=100)}}""", character_name="Jill")()

Stop program Write a story about a person and use \ {{character_name}} whenever you need to write their name:
STORY
 {{parse (gen max_tokens=100)}}

In [ ]:
import guidance
llm = guidance.llms.OpenAI(
    'text-davinci-003',
    api_type='azure',
    api_key='sk-...',
    api_base='https://example-endpoint.openai.azure.com',
    api_version='2023-05-15',
    deployment_id='...'
)

program = guidance("""My favorite flavor is{{gen 'flavor' max_tokens=10 stop="."}}""", llm=llm)
program()